In [119]:
def crs_prepossess(gpdf, init_crs, bfr_crs):
    gpdf_crs = gpdf.copy()
    if gpdf_crs.crs == None:
        gpdf_crs.crs = {'init': u'epsg:{}'.format(init_crs)}
    return gpdf_crs.to_crs(epsg=bfr_crs)

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    from math import radians, cos, sin, asin, sqrt
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    m = km *1000
    return m   


def ptfromln(pt, ln):
    """
    crs: espg: 4326, crs that uses lat lon
    get the projection of lonlat point to line
    then get distance on earth between point and line
    """
    n_pt = ln.interpolate(ln.project(pt))
    lon1, lat1 = n_pt.coords[0]
    lon2, lat2 = pt.coords[0]
    return haversine(lon1, lat1, lon2, lat2)

def pts2seg(gp_pts, gp_segs, init_crs=4326, bfr_crs=3559, near_dis_thres=5, buffer_dis=50):
    """
    pts and segs are assumed as geopandas.GeoDataFrame with crs:4326, which means (lon,lat) points
    1. check crs and change crs to epsg:3559 (NAD83(NSRS2007) / Maryland)
    2. get segid of near seg(s) based on var:near_dis_thres for each point
    3. for those points without any near segs
     - buffer them var:buffer_dis meters to find near segs
     - use func:ptfromln to get on earth distance from point to line
     - get one segid of the nearest seg
    """
    import geopandas as gp
    import pandas as pd


    gp_pts_crs = crs_prepossess(gp_pts,init_crs,bfr_crs)
    gp_segs_crs = crs_prepossess(gp_segs,init_crs,bfr_crs)

    gp_pts_crs_bfr = gp_pts_crs.copy()
    gp_pts_crs_bfr.geometry = gp_pts_crs_bfr.buffer(near_dis_thres)

    close_jn = gp.tools.sjoin(gp_pts_crs_bfr, gp_segs_crs)[['index_right']]

    processed_pts = set(pd.unique(close_jn.index))
    mask = (~gp_pts_crs_bfr.index.isin(processed_pts))
    far_jns = []
    while gp_pts_crs_bfr[mask].shape[0]!=0:
        gp_pts_crs_bfr.loc[mask, 'geometry'] = gp_pts_crs_bfr[mask].buffer(buffer_dis)
        jn = gp.tools.sjoin(gp_pts_crs_bfr[mask], gp_segs_crs)[['index_right']]
        far_jns.append(jn)
        processed_pts |= set(pd.unique(jn.index))
        mask = (~gp_pts_crs_bfr.index.isin(processed_pts))

    far_jns = pd.concat(far_jns)
    mr_far_jns = pd.merge(gp_segs[['geometry']],far_jns , left_index=True, right_on=['index_right'])
    mr_far_jns = pd.merge(gp_pts[['geometry']],mr_far_jns, left_index=True, right_index=True)
    mr_far_jns['dis']=mr_far_jns.apply(lambda x: ptfromln(x.geometry_x, x.geometry_y),axis=1)

    group = mr_far_jns.groupby(level=0).apply(lambda x: x.iloc[x.dis.values.argmin()][['index_right']])
    result = close_jn.append(group).reset_index()
    result.columns=['pt_index', 'seg_index']
    return result


In [97]:
import geopandas as gp
segs_path = '../data/opendc_segments.geojson'
segs = gp.read_file(segs_path)

In [113]:
from shapely.geometry import Point
lonlats =[(-77.0499797097, 38.9025337467), (-76.9750248217, 38.9342833419), (-77.0499797097, 38.9025337467), (-77.018353403, 38.8174155704),
 (-77.0499797097, 38.9025337467), (-77.0499797097, 38.9025337467), (-77.0499797097, 38.9025337467), (-77.0499797097, 38.9025337467),
 (-76.982369647, 38.9416450899), (-77.01466812, 38.9264066913), (-77.01466812, 38.9264066913), (-77.0731913537, 38.9241655819),
 (-77.0060749316, 38.9344875478), (-77.0172487613, 38.8206668992), (-77.1099421133, 38.9334457127), (-77.0248761622, 38.9299447811),
 (-77.091346861, 38.9414896542), (-76.9651641877, 38.9251762117), (-76.9651641877, 38.9251762117), (-76.9651641877, 38.9251762117),
 (-77.0085563587, 38.8724597385), (-77.0085563587, 38.8724597385), (-77.0499797097, 38.9025337467), (-77.1099421133, 38.9334457127),
 (-77.1099421133, 38.9334457127), (-76.9663809737, 38.9176546129), (-77.0142406682, 38.8920467534), (-77.0142406682, 38.8920467534),
 (-77.0004259836, 38.9641158389), (-76.9810398283, 38.9041776464), (-76.9926023075, 38.9380717842), (-77.0499797097, 38.9025337467),
 (-77.0499797097, 38.9025337467), (-76.9590017698, 38.8619306698), (-77.0367570316, 38.9834042588), (-77.018353403, 38.8174155704),
 (-77.018353403, 38.8174155704), (-77.018353403, 38.8174155704), (-76.9584917822, 38.9200200968), (-76.9833049061, 38.8533051238),
 (-76.9833049061, 38.8533051238), (-77.01466812, 38.9264066913), (-76.9584917822, 38.9200200968), (-77.0499797097, 38.9025337467),
 (-77.0085563587, 38.8724597385), (-77.0085563587, 38.8724597385), (-77.0060749316, 38.9344875478), (-77.0060749316, 38.9344875478),
]
print len(lonlats)
pts = [Point(lonlat) for lonlat in lonlats]
pts_gpdf = gp.GeoDataFrame(pts, columns=['geometry'])

48


In [120]:
pts_seg = pts2seg(pts_gpdf, segs)

In [124]:
pts_segids = pts_seg.merge(segs[['STREETSEGID']], left_on=['seg_index'], right_index=True)

In [125]:
pts_segids.sort('pt_index')

,pt_index,seg_index,STREETSEGID
0,0,2741,4361
56,1,8892,10080
1,2,2741,4361
10,3,11846,6993
2,4,2741,4361
3,5,2741,4361
4,6,2741,4361
5,7,2741,4361
14,8,7988,9879
15,9,5716,1405
